In [ ]:
import pandas as pd
from py2neo import Graph, Node, Relationship

d:\Anaconda_other\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
d:\Anaconda_other\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# 数据读取

neo4j 账号：neo4j 密码：jhc20021102

In [5]:
df_movie = pd.read_csv("../../数据集/豆瓣电影数据集/豆瓣电影数据集/movies.csv")
df_movie

,MOVIE_ID,NAME,ALIAS,ACTORS,COVER,DIRECTORS,DOUBAN_SCORE,DOUBAN_VOTES,GENRES,IMDB_ID,...,MINS,OFFICIAL_SITE,REGIONS,RELEASE_DATE,SLUG,STORYLINE,TAGS,YEAR,ACTOR_IDS,DIRECTOR_IDS
0,26670818,情定河州,情定临夏天使然,王博/吴佳尼/王姬/高丽雯/郭力行/尹哲/沈丹萍/罗中旭/臧金生/罗刚/居文沛/阎青妤,NaN,尹哲,0.0,0.0,剧情/爱情,NaN,...,0.0,NaN,中国大陆,NaN,RmNQQeyzb,电影《情定临夏天使然》讲述临夏新一代青年人发奋图强、借助国家一带一路战略励志创业的故事。,甘肃/临夏/伊斯兰/中国/2016/中国大陆/烂片/宣传伊斯兰教的电影,2049.0,王博:|吴佳尼:1313262|王姬:1275275|高丽雯:1325661|郭力行:135...,尹哲:1326188
1,25815002,我不是李小龙,NaN,谷尚蔚/吴孟达/曾志伟/杜海涛,NaN,洪金宝,0.0,0.0,动作/爱情,NaN,...,0.0,NaN,中国大陆,NaN,EZnVfiNYf,桀骜不驯的如龙武功高强，在一场比赛中，被打成重伤，被诊今生不能再用功夫。女友荆兰为激发他重新...,穿越/华语,2049.0,谷尚蔚:1330813|吴孟达:1016771|曾志伟:1002862|杜海涛:1313024,洪金宝:1055887
2,26392287,曼哈顿中国女孩,NaN,NaN,NaN,NaN,0.0,0.0,剧情,NaN,...,0.0,NaN,中国大陆 / 美国,NaN,NuUvEJnzb,平民女孩李莉只身初入曼哈顿求学，在历经迷失与困惑之后，凭借努力与善良收获了事业上的成功，同时...,NaN,2049.0,NaN,NaN
3,26695995,绿毛水怪,NaN,NaN,NaN,梁栋/吴国墉,0.0,0.0,爱情,NaN,...,0.0,NaN,中国大陆,NaN,rqaqyb6ea,王小波经典中篇小说《绿毛水怪》将改编电影。《绿毛水怪》是王小波早期手稿作品，以天马行空的想象...,小波/王小波/爱情/小说改编/文学改编/剧情/中国/2017,2049.0,NaN,梁栋:|吴国墉:
4,26392292,为了祖国,NaN,NaN,NaN,NaN,0.0,0.0,剧情/历史,NaN,...,0.0,NaN,中国大陆 / 韩国,NaN,Inrqy7uib,1932年上海虹口爆炸案后，韩国国父金九在褚辅成、朱爱宝这些普通群众的帮助下逃到嘉兴避难。金...,NaN,2049.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140497,5108106,Pretty Boy and Rosa - 电影,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,...,0.0,NaN,NaN,NaN,nN2zjz7Yf,NaN,NaN,0.0,NaN,NaN
140498,5108082,番茄杀手 - 电影,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,...,0.0,NaN,NaN,NaN,N2jjNYnVf,NaN,NaN,0.0,NaN,NaN
140499,5108078,Fast Track - 电影,NaN,基思·卡拉丹/弗雷德·威廉森/邓肯·雷格尔/布兰迪·莱德福特/塞巴斯蒂安·斯宾赛/罗伯特·卡...,NaN,T·J·斯科特,0.0,0.0,剧情/动作,tt0124239,...,60.0,NaN,美国,NaN,AIBeyzzVf,NaN,NaN,1997.0,NaN,NaN
140500,5108060,7-9-13 - 电影,NaN,NaN,NaN,NaN,0.0,0.0,喜剧,tt0124270,...,0.0,NaN,丹麦,NaN,EJ3zzbERe,NaN,NaN,1934.0,NaN,NaN


In [ ]:
# 字段映射字典